<a href="https://colab.research.google.com/github/shizoda/education/blob/main/machine_learning/cnn/cifar10_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNNでの画像分類

## CNNとは

CNN (Convolutional Neural Network，畳み込みニューラルネットワーク )は、特に画像や映像の認識，解析において高い性能を発揮するディープラーニングの一種です．CNNは，入力データから特徴を自動的に学習し，識別や分類を行う能力を持っています．以下に，CNNの構造，動作原理，主な用途について説明します．

<a title="Aphex34, CC BY-SA 4.0 &lt;https://creativecommons.org/licenses/by-sa/4.0&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Typical_cnn.png"><img width="512" alt="Typical cnn" src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/63/Typical_cnn.png/512px-Typical_cnn.png?20151217030420"></a>

### 主な構成要素
CNNはディープラーニングの中でも，畳み込み層やプーリング層といった独自の構造を持っていることが特徴です．以下では主な構成要素について説明します．

- 畳み込み層

畳み込み層は，画像内の局所的な特徴抽出を行います．隣接するピクセル間におけるエッジや色の変化といった局所的な特徴を検出し，画像内の情報を保持しつつ，高度な特徴抽出を実現します．

<a title="Michael Plotke, CC BY-SA 3.0 &lt;https://creativecommons.org/licenses/by-sa/3.0&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:2D_Convolution_Animation.gif"><img width="256" alt="2D Convolution Animation" src="https://upload.wikimedia.org/wikipedia/commons/1/19/2D_Convolution_Animation.gif?20130203224852"></a>

- プーリング層

プーリング層では，畳み込み層で抽出された特徴が移動しても影響を受けないようにする役割を担います．畳み込み層から出力される特徴は局所的なものです．対象の特徴を維持しながら位置に関する情報をそぎ落とすことで，重要な情報のみを保持し，特徴量のサイズを小さくすることができます．

最大プーリングと平均プーリングが有名です．

<a title="Muhamad Yani, et al.; Creative Commons Attribution 3.0 Unported &lt;https://creativecommons.org/licenses/by/3.0/&gt;" href="https://www.researchgate.net/figure/Illustration-of-Max-Pooling-and-Average-Pooling-Figure-2-above-shows-an-example-of-max_fig2_333593451"><img width = "256" alt ="Creative Commons Attribution 3.0 Unported" src="https://www.researchgate.net/publication/333593451/figure/fig2/AS:765890261966848@1559613876098/Illustration-of-Max-Pooling-and-Average-Pooling-Figure-2-above-shows-an-example-of-max.png"></a>

- 全結合層

畳み込み層やプーリング層で抽出された特徴を基に，最終的な分類や予測を行う層です．分類であれば，入力データがどのクラスに属するかを決定します．

## CNN での画像分類

ここでは単純な CNN モデルを画像分類用にトレーニングし、評価します。

データセットとして [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) を用います．CIFAR-10 は、10 の異なるクラス（飛行機、自動車、鳥、猫など）に属する全 60,000 枚の小さなカラー画像（32x32ピクセル）を含むデータセットです。

**実行する場合には、Google Colab の GPU ランタイムをオンにしてください**

<img src="https://raw.githubusercontent.com/shizoda/education/main/machine_learning/cnn/runtime1.png" height="300"> <img src="https://raw.githubusercontent.com/shizoda/education/main/machine_learning/cnn/runtime2.png" height="300">

### PyTorch のインポート

深層学習のための主要なフレームワークとして、Google が開発した TensorFlow と、Meta (Facebook) が開発した PyTorch が有名です。今回は PyTorch を使用します。

<a title="PyTorch, BSD &lt;http://opensource.org/licenses/bsd-license.php&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:PyTorch_logo_black.svg"><img width="256" alt="PyTorch logo black" src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c6/PyTorch_logo_black.svg/256px-PyTorch_logo_black.svg.png?20200318230141"></a>

前述のとおり、高速な並列演算のために GPU を使用します。使用できる状態であることを確認しています。

In [ ]:
# PyTorch 関連のライブラリをインポートします
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split

# GPU が利用可能であることを確認
if not torch.cuda.is_available():
    print("GPU is not available. Please change runtime type to 'GPU' in the Runtime menu.")

### データのロード

CIFAR-10データセットをダウンロードします。学習・検証・テストの各データセットとなります。

- `transforms.Compose` は、データに適用する一連の前処理手順を定義します。この例では、画像をPyTorchテンソルに変換し、正規化を行います。

- `DataLoader` は、データのバッチ処理、シャッフル、多重プロセスによるデータの読み込みを扱います。

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# 学習用データセットをロードし、検証用データセットに分割します。
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_size = int(0.8 * len(trainset))
validation_size = len(trainset) - train_size
train_dataset, validation_dataset = random_split(trainset, [train_size, validation_size])

# バッチサイズを100とし、学習用データローダと検証用データローダを定義します。
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
validationloader = torch.utils.data.DataLoader(validation_dataset, batch_size=100, shuffle=False)

# テスト用データセットをロードします。
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 画像を表示するためのヘルパー関数
def imshow(img):
    img = img / 2 + 0.5     # 正規化を元に戻す
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# DataLoader から画像を取得して表示する関数
def show_images(dataloader, num_images):
    dataiter = iter(dataloader)
    images, labels = next(dataiter)  # バッチを一つ取得

    # 画像を表示
    imshow(torchvision.utils.make_grid(images[:num_images]))
    # 正解ラベルを表示
    print(' '.join(f'{classes[labels[j]]:5s}' for j in range(num_images)))

# 使用例：trainloader から 4 枚の画像を表示
show_images(trainloader, 4)


### CNN モデルの定義

`forward` メソッドを１行ずつ確認して、どのような構成になっているか確認してみましょう。

畳み込み層の後には ReLu という活性化関数を入れています。[活性化関数って？ (Zenn)](https://zenn.dev/nekoallergy/articles/ml-basic-act-01)

In [ ]:
# Net クラスは、CNNモデルのアーキテクチャを定義します。
# モデルは畳み込み層（nn.Conv2d）、プーリング層（nn.MaxPool2d）、全結合層（nn.Linear）から構成されています。
# forward メソッドは、ネットワークを通じて入力がどのように進むかを定義します

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):

        # 畳み込み層１
        x = self.conv1(x)
        x = F.relu(x) # 活性化関数

        # プーリング層
        x = self.pool(x)

        # 畳み込み層２
        x = self.conv2(x)
        x = F.relu(x) # 活性化関数

        # プーリング層２
        x = self.pool(x)

        # 全結合層
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

# インスタンス化。Net クラス（設計図）のオブジェクト（実体）を net とする
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net = Net().to(device)

### 3. 損失関数とオプティマイザを定義する

誤差関数（nn.CrossEntropyLoss）は、モデルの予測と実際のラベル間の差異を測定します。
最適化アルゴリズム（optim.SGD）は、モデルのパラメータを調整するために使用されます。

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### ネットワークの学習

学習データセットから画像をランダムに数枚取り出したミニバッチ (mini-batch) をネットワークに入力すると，何らかの出力が得られます．それを正解と比較し，上記の誤差関数によって評価します．

これを繰り返していく中で，毎回少しずつ誤差関数を小さくするようにネットワークの重み（畳み込みに用いるカーネル）を更新していきます．徐々に，画像を入力すると正解が得られるようになってきます．

In [ ]:
# 検証用データセットに対する損失が5エポック連続で改善しない場合、学習を終了します。
best_val_loss = float("inf")
patience = 5
trigger_times = 0

train_losses = []
val_losses = []

for epoch in range(10):  # データセットを複数回ループ
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # if i % 2000 == 1999:    # 2000ミニバッチごとに出力
        #    print('[%d, %5d] loss: %.3f' %
        #          (epoch + 1, i + 1, running_loss / 2000))
        #    running_loss = 0.0

    # 検証用データセットに対する損失を計算
    val_loss = 0.0
    net.eval()
    with torch.no_grad():
        for data in validationloader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)

            outputs = net(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    train_losses.append(running_loss / len(trainloader))
    val_losses.append(val_loss / len(validationloader))

    # 損失の値をグラフとして表示
    plt.clf()
    plt.plot(train_losses, label='Training loss')
    plt.plot(val_losses, label='Validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.xlim(left=0)
    plt.ylim(bottom=0)
    plt.legend()
    plt.show()
    print("Train loss     : ", running_loss / len(trainloader))
    print("Validation loss: ", val_loss / len(validationloader))

    # 検証用データセットに対する損失が改善しないエポックが続いたら学習を終了
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        trigger_times = 0
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print(f"Early stopping at epoch {epoch}")
            break

print('Finished Training')

### テスト
訓練されたモデルを使用してテストデータセット上でのパフォーマンスを評価します。
最終的な精度（正確に分類された画像の割合）を求めることができます。

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)

        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
# テストデータの画像を表示し、正解と推定値を表示する関数
def visualize_test_predictions(start_idx, end_idx):
    testloader = torch.utils.data.DataLoader(testset, batch_size=1,
                                             shuffle=False, num_workers=2)
    images, labels = zip(*[(data[0], data[1]) for i, data in enumerate(testloader) if start_idx <= i < end_idx])

    # モデルの予測
    predicted_labels = []
    with torch.no_grad():
        for image in images:
            image = image.to(device)
            outputs = net(image)
            _, predicted = torch.max(outputs, 1)
            predicted_labels.append(predicted.item())

    # 画像とラベルを表示
    for i in range(len(images)):
        print(f"Image {start_idx + i}")
        imshow(torchvision.utils.make_grid(images[i]))
        print(f"Correct label: {classes[labels[i]]}")
        print(f"Predicted label: {classes[predicted_labels[i]]}\n")

# 例: インデックス範囲 0 から 5 の画像を表示
visualize_test_predictions(0, 5)
